In [1]:
import anndata as ad
import numpy as np
import scipy
import pandas as pd
import scanpy as sc


#---------------------------------------------------------Load the matrixs-----------------------------------------------------

normed_counts = "/Users/apple/Desktop/KB/Dataset1/stateFate_inVitro_normed_counts.mtx.gz"  #snakemake.input['normed_counts']
gene_names = "/Users/apple/Desktop/KB/Dataset1/stateFate_inVitro_gene_names.txt.gz" #snakemake.input['gene_names']
clone_matrix = "/Users/apple/Desktop/KB/Dataset1/stateFate_inVitro_clone_matrix.mtx.gz" #snakemake.input['clone_matrix']
metadata = "/Users/apple/Desktop/KB/Dataset1/stateFate_inVitro_metadata.txt.gz" #snakemake.input['metadata']

# load data
normed_counts_mat = scipy.io.mmread(normed_counts).tocsr()
genes = pd.read_csv(gene_names, sep='\t',header=None).to_numpy().flatten()
clone_mat = scipy.io.mmread(clone_matrix).tocsr()
meta_df = pd.read_csv(metadata, sep='\t')


#-------------------------------Get num_genes of highly expressed genes from the orginal data(all cells)--------------------
# create full adata
adata = ad.AnnData(normed_counts_mat, obs=meta_df, var=pd.DataFrame(index=genes), dtype=np.float32)
# optimize dtypes
adata.obs['Library'] = adata.obs['Library'].astype('category')
adata.obs['Time point'] = adata.obs['Time point'].astype(int)
adata.obs['Starting population'] = adata.obs['Starting population'].astype('category')
adata.obs['Cell type annotation'] = adata.obs['Cell type annotation'].astype('category')
adata.obs['Well'] = adata.obs['Well'].astype(int)
# assign clone_id
adata.obs['clone_id'] = (clone_mat @ np.arange(1,1+clone_mat.shape[1])) - 1
print("number of lineages: ", len(adata.obs['clone_id'].unique()))

# get 2000 genes from the 130887(all) cells
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata,n_top_genes=2000)


#---------------------------------Creat the subset adata with trimmed number of genes and number of cells--------------------
# create full adata
adata_cp = ad.AnnData(normed_counts_mat, obs=meta_df, var=pd.DataFrame(index=genes), dtype=np.float32)

# optimize dtypes
adata_cp.obs['Library'] = adata_cp.obs['Library'].astype('category')
adata_cp.obs['Time point'] = adata_cp.obs['Time point'].astype(int)
adata_cp.obs['Starting population'] = adata_cp.obs['Starting population'].astype('category')
adata_cp.obs['Cell type annotation'] = adata_cp.obs['Cell type annotation'].astype('category')
adata_cp.obs['Well'] = adata_cp.obs['Well'].astype(int)
# assign clone_id
adata_cp.obs['clone_id'] = (clone_mat @ np.arange(1,1+clone_mat.shape[1])) - 1

# remove the cells that belong to the clone_id with few cells
value_counts = adata.obs['clone_id'].value_counts()
frequency_dict = {}
for value, count in value_counts.items():
    if count in frequency_dict:
        frequency_dict[count].append(value)
    else:
        frequency_dict[count] = [value]

clone_for_remove = frequency_dict[81585]+frequency_dict[2]+frequency_dict[3]+frequency_dict[4] 
adata_subset = adata_cp[~adata_cp.obs['clone_id'].isin(clone_for_remove)]
print("adata_subset.obs.shape:", adata_subset.obs.shape)

# trim the number of genes to the required number using the highly variable gene calculated from the original adata (all cells)
hvgene = (adata.var.highly_variable[adata.var.highly_variable==True]).index
print("number of the highly variable genes:", len(hvgene))
adata_subset = adata_subset[:,hvgene]
print("adata_subset.X.shape:", adata_subset.X.shape)

/opt/anaconda3/envs/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/opt/anaconda3/envs/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


number of lineages:  5865


/opt/anaconda3/envs/scvi-env/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/opt/anaconda3/envs/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/opt/anaconda3/envs/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


adata_subset.obs.shape: (41201, 9)
number of the highly variable genes: 2000
adata_subset.X.shape: (41201, 2000)


In [6]:
# save the data to h5ad file
adata_subset.write('Larry_41201_2000.h5ad')